In [33]:
import sys
import subprocess
subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "pip"])
subprocess.check_call([sys.executable, "-m", "pip", "install", "requests"])

0

In [34]:
subprocess.check_call([sys.executable, "-m", "pip", "install", "beautifulsoup4",])

0

In [35]:


subprocess.check_call([sys.executable, "-m", "pip", "install", "pandas"])


0

In [30]:
import requests
import time
import random
import json
import re # regular expressions (pattern matching)
from bs4 import BeautifulSoup
import pandas as pd

In [4]:
# Base URL
URL="https://ie.trustpilot.com/review/brownthomas.com"

In [5]:
HEADERS = {
    "User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/18.5 Safari/605.1.15",
    "accept-language":"en-GB,en-US;q=0.9,en;q=0.8"
}

In [6]:
# setting constants
MAX_REVIEWS = 200
MAX_PAGES = 50 #safety
SLEEP_RANGE = (2.0,4.0)

In [7]:
# function to get response from URL
def fetch_page(url):
    resp = requests.get(url, headers=HEADERS, timeout=30)
    resp.raise_for_status() # raises httperror, if one occurred.
    return resp.text

In [8]:
# def test_fetch_page(url)
html = fetch_page(URL)

In [9]:
# returns a list of jason blobs

def find_json_blobs(html):
    """Return a list of parsed JSON blobs found in <script> tags."""
    soup = BeautifulSoup(html, "html.parser")
    blobs = []

    # 1) Next.js hydration data (often contains reviews)
    nd = soup.find("script", id="_NEXT_DATA_")
    if nd and nd.string:
        try:
            blobs.append(json.loads(nd.string))
        except Exception:
            pass

    # 2) Any structured ld+json blocks (may include a subset of reviews)
    for tag in soup.find_all("script", {"type": "application/ld+json"}):
        if tag.string:
            try:
                data = json.loads(tag.string)
                blobs.append(data)
            except Exception:
                # some pages include multiple JSON objects concatenated; try to split safely
                for m in re.finditer(r"\{.*?\}", tag.string, flags=re.DOTALL):
                    try:
                        blobs.append(json.loads(m.group(0)))
                    except Exception:
                        pass
    return blobs

In [10]:
# get all the data out of the objects
def flatten(obj):
    if isinstance(obj, dict):
        yield obj
        for v in obj.values():
            yield from flatten(v)
    elif isinstance(obj, list):
        for v in obj:
            yield from flatten(v)

In [11]:
# fnd keys that are relevant to reviews
def looks_like_review(d):
    return isinstance(d, dict) and (
        "rating" in d and "text" in d and "consumer" in d
    )


In [13]:
# extracting the data (normalize review)
def extract_review_fields(d):
    consumer = d.get("consumer") or {}
    dates = d.get("dates") or {}

    author_id = consumer.get("id")
    title = (d.get("title") or "").strip() if isinstance(d.get("title"), str) else None
    review_text = (d.get("text") or "").strip() if isinstance(d.get("text"), str) else None
    rating = d.get("rating")
    created_date = dates.get("publishedDate") or dates.get("submittedDate")

    if isinstance(rating, dict):
        rating = rating.get("value") or rating.get("ratingValue")

    row = {
        "author_id": author_id,
        "title": title,
        "review_text": review_text,
        "rating": rating,
        "created_date": created_date,
    }

    print(row)  # debug
    return row


In [14]:
# extract review from jason blobs
def parse_reviews_from_html(html):
    reviews = []

    # JSON-first approach
    for blob in find_json_blobs(html):
        for node in flatten(blob):
            if looks_like_review(node):
                print("yes")
                reviews.append(extract_review_fields(node))
                

            # Some pages store reviews under explicit arrays named 'reviews'
            if isinstance(node, dict):
                for k, v in node.items():
                    if k.lower() in ("reviews", "serviceReviews", "recentReviews") and isinstance(v, list):
                        for item in v:
                            if isinstance(item, dict) and looks_like_review(item):
                                reviews.append(extract_review_fields(item))

# Deduplicate by (text, date, author) heuristic
    seen = set()
    unique = []
    for r in reviews:
        key = (r["review_text"], r["created_date"], r["author_id"])
        if key not in seen and (r["review_text"] or r["title"]):
            unique.append(r)
            seen.add(key)

    return unique
  

In [15]:
parse_reviews_from_html(html)

[]

In [16]:
# loop for getting pages from the website
def get_reviews(base_url=URL, max_reviews=MAX_REVIEWS):
    out = []
    for page in range(1, MAX_PAGES + 1):
        url = base_url if page == 1 else f"{base_url}?page={page}"
        html = fetch_page(url)
        page_reviews = parse_reviews_from_html(html)

        # Fix: match the key names
        existing_texts = {r["review_text"] for r in out if r["review_text"]}
        fresh = [r for r in page_reviews if (r["review_text"] and r["review_text"] not in existing_texts)]
        out.extend(fresh)

        print(f"Page {page}: got {len(fresh)} new, total={len(out)}")
        if len(out) >= max_reviews:
            return out[:max_reviews]

        time.sleep(random.uniform(*SLEEP_RANGE))

    return out[:max_reviews]


In [17]:
reviews=get_reviews()


Page 1: got 0 new, total=0
Page 2: got 0 new, total=0


KeyboardInterrupt: 

In [ ]:
# unit tests verify that your function does what it is supposed to do
# so if my function is supposed to add two numbers together
# I can write unit tests like this
# 3+3 = 6 
# def add()
# test_add()
# actual result = add(3,3)
# expected result = 6
# if actual result == expected result then the test passes
# else the test fails

In [31]:
# Headers
HEADER = {
    "accept": "*/*",
    "accept-encoding": "gzip, deflate, br, zstd",
    "accept-language": "en-GB,en-US;q=0.9,en;q=0.8",
    "baggage": "sentry-environment=Production,sentry-release=businessunitprofile-consumersite%402.4845.0,sentry-public_key=7ac98d0742b24421b3d38448c4bf1184,sentry-trace_id=877b8e1f2533423cbdc92d5ea699cecd",
    "cookie": "TP.uuid=0706ee8f-76f5-4442-b3a2-b3e3700ea380; OptanonAlertBoxClosed=2025-09-08T15:22:09.409Z; ajs_anonymous_id=098d2a00-1d3a-4d88-b314-85b77407ce79; amplitude_idundefinedtrustpilot.com=eyJvcHRPdXQiOmZhbHNlLCJzZXNzaW9uSWQiOm51bGwsImxhc3RFdmVudFRpbWUiOm51bGwsImV2ZW50SWQiOjAsImlkZW50aWZ5SWQiOjAsInNlcXVlbmNlTnVtYmVyIjowfQ==; lc_referrer_url=https%3A%2F%2Fwww.google.com%2F; fc_referrer_url=https%3A%2F%2Fwww.google.com%2F; _hjHasCachedUserAttributes=true; __adroll_fpc=090700da94544d0729382867614e2fb7-1757344931547; _fbp=fb.1.1757344932562.32325800100083557; _hjSessionUser_391767=eyJpZCI6Ijg0YmMxMjY4LTUzNzEtNWI0Mi1iNjg2LWM5NDVmODZlNDQ1NiIsImNyZWF0ZWQiOjE3NTczNDQ5Mjk2NTYsImV4aXN0aW5nIjp0cnVlfQ==; _hjSessionUser_398288=eyJpZCI6ImYxNGQxOGEwLWY5YjEtNTk0NS05YTAyLWQxY2FhNzdiNGVkOSIsImNyZWF0ZWQiOjE3NTc0MDk2ODcxMzgsImV4aXN0aW5nIjp0cnVlfQ==; _biz_uid=15963d115de44399b6007b6572bc93a7; _gid=GA1.2.201371872.1757409687; _biz_flagsA=%7B%22Version%22%3A1%2C%22ViewThrough%22%3A%221%22%2C%22XDomain%22%3A%221%22%7D; _biz_nA=3; _biz_pendingA=%5B%5D; _ga_3TEL80JZSG=GS2.1.s1757409687$o1$g1$t1757410655$j52$l0$h0; amplitude_id_cfe705a69359b8a4c0049d061ee5787btrustpilot.com=eyJkZXZpY2VJZCI6IjBmMzNhMWNmLTc2OWEtNDIwZi04ZDU5LWYwOWYxYjhiZDFiNFIiLCJ1c2VySWQiOm51bGwsIm9wdE91dCI6ZmFsc2UsInNlc3Npb25JZCI6MTc1NzQxMDA2NzMyOSwibGFzdEV2ZW50VGltZSI6MTc1NzQxMTYyODc1OSwiZXZlbnRJZCI6MiwiaWRlbnRpZnlJZCI6MSwic2VxdWVuY2VOdW1iZXIiOjN9; _csrf=GSISE9yn_UcknXuQ4tWofQV_; _ga=GA1.1.909402240.1757344931; _gcl_au=1.1.1745043198.1757344930.1881067649.1757412719.1757412718; tp-external-referrer=%7B%22referrer%22%3A%22https%3A%2F%2Fwww.google.com%2F%22%2C%22entry%22%3A%22https%3A%2F%2Fie.trustpilot.com%2F%22%7D; OptanonConsent=isGpcEnabled=0&datestamp=Tue+Sep+09+2025+14%3A55%3A17+GMT%2B0100+(Irish+Standard+Time)&version=202503.1.0&browserGpcFlag=0&isIABGlobal=false&hosts=&consentId=27ed8391-26b9-4508-a0c3-079183ae230c&interactionCount=1&isAnonUser=1&landingPath=NotLandingPage&groups=C0001%3A1%2CC0002%3A1%2CC0003%3A1%2CC0004%3A1&intType=1&geolocation=IE%3BL&AwaitingReconsent=false; __ar_v4=WB2AUOFO2ZFVFHJ5MEHUWO%3A20250908%3A16%7CIUNG4UKUTNBEXHEW2IT6BS%3A20250908%3A16%7C6U7APKK4E5F45KVW4MS4AL%3A20250908%3A16; _hjSession_391767=eyJpZCI6IjExNjc1NDg0LTJlODMtNGMzZS04MWIyLTliZTZmYTRlZWU5NSIsImMiOjE3NTc0MjcxNTg3MzMsInMiOjAsInIiOjAsInNiIjowLCJzciI6MCwic2UiOjAsImZzIjowLCJzcCI6MH0=; amplitude_id_67f7b7e6c8cb1b558b0c5bda2f747b07trustpilot.com=eyJkZXZpY2VJZCI6IjA5OGQyYTAwLTFkM2EtNGQ4OC1iMzE0LTg1Yjc3NDA3Y2U3OSIsInVzZXJJZCI6bnVsbCwib3B0T3V0IjpmYWxzZSwic2Vzc2lvbklkIjoxNzU3NDI4NjM0OTY3LCJsYXN0RXZlbnRUaW1lIjoxNzU3NDI5MzU2Njk3LCJldmVudElkIjo1NSwiaWRlbnRpZnlJZCI6OCwic2VxdWVuY2VOdW1iZXIiOjYzfQ==; _ga_11HBWMC274=GS2.1.s1757425910$o8$g1$t1757429356$j31$l0$h0",
    "newrelic": "eyJ2IjpbMCwxXSwiZCI6eyJ0eSI6IkJyb3dzZXIiLCJhYyI6IjQzODU4ODciLCJhcCI6IjUzODY4MjQ5OSIsImlkIjoiMmQxZmI0ODJjZTdlOWI4YSIsInRyIjoiNzUwMWEzZjVkZDliZWZhY2MyOTA3ODA4NWFkNThmNjEiLCJ0aSI6MTc1NzQyOTM4MTE5MX19",
    "priority": "u=1, i",
    "referer": "https://ie.trustpilot.com/review/www.asos.com?page=2",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin",
    "sentry-trace": "877b8e1f2533423cbdc92d5ea699cecd-aa8c8d2f9d573f99",
    "traceparent": "00-7501a3f5dd9befacc29078085ad58f61-2d1fb482ce7e9b8a-01",
    "tracestate": "4385887@nr=0-1-4385887-538682499-2d1fb482ce7e9b8a----1757429381191",
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/18.5 Safari/605.1.15",
    "x-nextjs-data": "1"
}


all_reviews = []

# Target: ASOS reviews, pages 2 to 15
target = 'www.asos.com'

for i in range(2, 15):  # pages 2 to 15
    url = f'https://ie.trustpilot.com/_next/data/businessunitprofile-consumersite-2.4845.0/review/www.asos.com.json?page={i}&businessUnit={target}'
    
    resp = requests.get(url, headers=HEADER)
    if resp.status_code != 200:
        print(f"Failed to fetch page {i}")
        continue
    
    res = resp.json()
    
    # Extract reviews from JSON
    reviews = res.get('pageProps', {}).get('reviews', [])
    all_reviews.extend(reviews)
    
    print(f'Page {i} fetched, {len(reviews)} reviews.')
    
    time.sleep(1)  # polite delay to avoid being blocked

# Normalize into DataFrame
df = pd.json_normalize(all_reviews)
df.to_csv('asos_reviews.csv', index=False)

print(f"Saved {len(all_reviews)} reviews to CSV")


Page 2 fetched, 20 reviews.
Page 3 fetched, 20 reviews.
Page 4 fetched, 20 reviews.
Page 5 fetched, 20 reviews.
Page 6 fetched, 20 reviews.
Page 7 fetched, 20 reviews.
Page 8 fetched, 20 reviews.
Page 9 fetched, 20 reviews.
Page 10 fetched, 20 reviews.
Page 11 fetched, 20 reviews.
Page 12 fetched, 20 reviews.
Page 13 fetched, 20 reviews.
Page 14 fetched, 20 reviews.
Saved 260 reviews to CSV


In [ ]:
# load CSV and print head
df = pd.read_csv('asos_reviews.csv')
print(df.head())

                         id  filtered  pending  \
0  68b951e16796366d612c0a5a     False    False   
1  68b9508a183d3a0e366f4055     False    False   
2  68b93c80ca0e2a5da9e471f2     False    False   
3  68b8afadbca99a540889a0eb     False    False   
4  68b88f6f5a626c5041eea2c4     False    False   

                                                text  rating  \
0  I ordered tshirt of 83£ and did a partial retu...       1   
1  AWFUL! Still waiting for a refund that was app...       1   
2  Love asos fast delivery and selection - but th...       4   
3  Dreadful customer care. My order hasn’t arrive...       1   
4  I ordered a baseball cap from ASOS, as I am a ...       1   

                                      title  likes   source  report  \
0  Ordered tshirt of 83£ and did a partial…      0  Organic     NaN   
1                                     AWFUL      0  Organic     NaN   
2                          Great selection       0  Organic     NaN   
3                    Dreadful 

In [ ]:
df

,id,filtered,pending,text,rating,title,likes,source,report,hasUnhandledReports,...,dates.publishedDate,dates.updatedDate,dates.submittedDate,consumer.id,consumer.displayName,consumer.imageUrl,consumer.numberOfReviews,consumer.countryCode,consumer.hasImage,consumer.isVerified
0,68b951e16796366d612c0a5a,False,False,I ordered tshirt of 83£ and did a partial retu...,1,Ordered tshirt of 83£ and did a partial…,0,Organic,NaN,False,...,2025-09-04T10:46:25.000Z,NaN,NaN,686e4d994e662d62b6628580,MZ Iqbal,https://user-images.trustpilot.com/686e4d994e6...,2,GB,True,False
1,68b9508a183d3a0e366f4055,False,False,AWFUL! Still waiting for a refund that was app...,1,AWFUL,0,Organic,NaN,False,...,2025-09-04T10:40:42.000Z,NaN,NaN,5be5c5715483f4b8a9eca376,Danielle,NaN,68,GB,False,False
2,68b93c80ca0e2a5da9e471f2,False,False,Love asos fast delivery and selection - but th...,4,Great selection,0,Organic,NaN,False,...,2025-09-04T09:15:12.000Z,NaN,NaN,68b93b6acc7c940c0350ce77,Elle,https://user-images.trustpilot.com/68b93b6acc7...,3,GB,True,False
3,68b8afadbca99a540889a0eb,False,False,Dreadful customer care. My order hasn’t arrive...,1,Dreadful customer care,0,Organic,NaN,False,...,2025-09-03T23:14:21.000Z,NaN,NaN,53ed24dd000064000184a7b7,"Charlotte, Swansea",NaN,8,GB,False,False
4,68b88f6f5a626c5041eea2c4,False,False,"I ordered a baseball cap from ASOS, as I am a ...",1,I ordered a baseball cap from ASOS,0,Organic,NaN,False,...,2025-09-03T20:56:48.000Z,NaN,NaN,5d0922439522f8117094f6bd,Matty Edmond,https://user-images.trustpilot.com/5d092243952...,2,GB,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,6888d772e7a1c201b376a0b5,False,False,"Awful customer service , very rude. Ended chat...",1,Awful customer service,1,Organic,NaN,False,...,2025-07-29T16:15:14.000Z,NaN,NaN,6888d76bb7b8170997614ac9,Jessica hard,NaN,1,GB,False,False
256,6888c28c5518912ae4a4a82f,False,False,"If I could give 0 stars I would, I have been s...",1,Absolutely disgusting company,0,Organic,NaN,False,...,2025-07-29T14:46:04.000Z,NaN,NaN,6888c28364d5cf79467d2588,Marilyn Trotman,NaN,1,GB,False,False
257,6888ba548b4bdbeb9df0cf73,False,False,I am absolutely disgusted with Asos and their ...,1,Account closed without notification/customer s...,0,Organic,NaN,False,...,2025-07-29T14:11:00.000Z,2025-07-30T14:43:27.000Z,NaN,5b2e54494de5666d34a91eaf,Michelle Berry,NaN,28,GB,False,False
258,6888aaff75aec114fdf44be8,False,False,"Like (many) others, I have experienced appalli...",1,Appalling customer service,0,Organic,NaN,False,...,2025-07-29T13:05:36.000Z,NaN,NaN,65db9497b4c250001299a491,Kim,NaN,4,GB,False,False


In [46]:
# print number of rows and collumns
print(df.shape)

(260, 34)


In [ ]:
# print collumns name
print(df.columns.tolist())

['id', 'filtered', 'pending', 'text', 'rating', 'title', 'likes', 'source', 'report', 'hasUnhandledReports', 'reply', 'consumersReviewCountOnSameDomain', 'consumersReviewCountOnSameLocation', 'productReviews', 'language', 'location', 'labels.merged', 'labels.verification.isVerified', 'labels.verification.createdDateTime', 'labels.verification.reviewSourceName', 'labels.verification.verificationSource', 'labels.verification.verificationLevel', 'labels.verification.hasDachExclusion', 'dates.experiencedDate', 'dates.publishedDate', 'dates.updatedDate', 'dates.submittedDate', 'consumer.id', 'consumer.displayName', 'consumer.imageUrl', 'consumer.numberOfReviews', 'consumer.countryCode', 'consumer.hasImage', 'consumer.isVerified']


In [ ]:
# print data type and Null values
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260 entries, 0 to 259
Data columns (total 34 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   id                                      260 non-null    object 
 1   filtered                                260 non-null    bool   
 2   pending                                 260 non-null    bool   
 3   text                                    260 non-null    object 
 4   rating                                  260 non-null    int64  
 5   title                                   260 non-null    object 
 6   likes                                   260 non-null    int64  
 7   source                                  260 non-null    object 
 8   report                                  0 non-null      float64
 9   hasUnhandledReports                     260 non-null    bool   
 10  reply                                   0 non-null      float6

In [47]:
df_clean = df.copy()
